In [4]:
import requests
from bs4 import BeautifulSoup as bs

Take in a URL

In [6]:
url = 'https://www.allrecipes.com/recipe/24074/alysias-basic-meat-lasagna/'
page = requests.get(url)
soup = bs(page.content, "html.parser")

In [20]:
title = soup.find("h1", class_="headline heading-content elementFont__display").text

In [49]:
recipe_info = soup.find_all("div", class_="recipe-meta-item-header elementFont__subtitle--bold elementFont__transformCapitalize")
recipe_info = [x.text for x in recipe_info]
recipe_info_details = soup.find_all("div", class_="recipe-meta-item-body elementFont__subtitle")
recipe_info_details = [x.text[:-1] for x in recipe_info_details]

recipe_info = zip(recipe_info, recipe_info_details)
for item in recipe_info:
  print(item)

('prep:', '50 mins')
('cook:', '45 mins')
('total:', '1 hr 35 mins')
('Servings:', '10')
('Yield:', '10 servings')


In [36]:
ingredients = soup.find_all("span", class_="ingredients-item-name elementFont__body")
ingredients = [x.text for x in ingredients]
# need to translate some encodings and special characters
# also need to learn how to distinguish ingredients and measurements

In [40]:
steps = soup.find_all("div", class_="section-body elementFont__body--paragraphWithin elementFont__body--linkWithin")
steps = [x.text[:-2] for x in steps]

In [47]:
nutrition = soup.find("div", class_="recipeNutritionSectionBlock")
nutrition = nutrition.text[1:-17]

nutrition

'Per Serving:  567 calories; protein 36g; carbohydrates 35.6g; fat 31.4g; cholesterol 129.9mg; sodium 943.1mg.'